# Import Library

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import json
import os

# Data Load

In [5]:
path = 'c:/research_persona/sentiment_analysis/movie_review_sentiment'
df = pd.read_csv(path + '/raw_data_nsmc.csv')

d_path = 'd:/naver_review/'
review_folder = os.listdir(d_path)
t = {}
for ix, review_file in enumerate(review_folder):
    with open(d_path + review_file, 'rb') as f:
        t[ix] = json.load(f)
t1 = pd.DataFrame.from_dict(t[0])
t2 = pd.DataFrame.from_dict(t[1])
t3 = pd.DataFrame.from_dict(t[2])
t4 = pd.DataFrame.from_dict(t[3])
t5 = pd.DataFrame.from_dict(t[4])
t6 = pd.DataFrame.from_dict(t[5])
df2 = pd.concat((t1, t2, t3, t4, t5, t6))
del t, t1, t2, t3, t4, t5, t6

In [6]:
df.shape, df2.shape

((712383, 8), (2532415, 10))

# Sentiment Dictionary Load

In [8]:
from ksenticnet_kaist import *

ksenticnet = get_ksenticnet()
sentiwordknu = pd.read_excel('../감정단어사전0603.xlsx')

In [13]:
len(ksenticnet), sentiwordknu.shape

(5465, (428, 6))

# 다중 감정 라벨링
- 직접 라벨링 (비교용 1,000개)
- JST (Topic Modeling, un-supervised)
- sentiment dictionary mapping

# Text 전처리

# 감정 단어 사전 분석

In [53]:
keys = list(ksenticnet.keys())

In [54]:
senticvals = [[float(i) for i in val[:4]] for val in  ksenticnet.values()]

In [66]:
sentiments = []
polarity = []
semantics = []
for key, val in ksenticnet.items():
    for i in val[4:]:
        if i in ['positive', 'negative']:
            polar_ind = val.index(i)
            sentiments.append(val[4 : polar_ind])
            polarity.append(val[polar_ind : polar_ind+2])
            semantics.append(val[polar_ind+2 :])
            break

In [81]:
set([j for i in sentiments for j in i])

{'#admiration',
 '#anger',
 '#disgust',
 '#fear',
 '#interest',
 '#joy',
 '#sadness',
 '#surprise'}

In [75]:
print(sentiwordknu.감정범주.unique(), sentiwordknu.감정범주.nunique())

['혐오' '슬픔' '기쁨' '중성' '흥미' '분노' '지루함' '놀람' '공포' '기타' '통증'] 11


```
'#admiration'    감탄
'#anger'         분노
'#disgust'       싫음
'#fear'          두려움
'#interest'      흥미
'#joy'           즐거움
'#sadness'       슬픔
'#surprise'      놀라움
```

In [84]:
set([len(i) for i in sentiments])

{2}

In [88]:
senticvals

[[0.0, 0.124, -0.05, 0.203],
 [-0.003, 0.236, 0.009, 0.147],
 [0.0, 0.822, 0.0, 0.712],
 [0.045, 0.0, 0.033, 0.089],
 [0.0, 0.921, 0.0, 0.993],
 [0.791, 0.857, 0.0, 0.0],
 [0.514, -0.97, 0.0, 0.899],
 [-0.9, 0.0, -0.874, 0.0],
 [-0.373, 0.0, 0.14, -0.911],
 [-0.05, 0.0, -0.11, -0.04],
 [0.0, 0.0, -0.855, -0.755],
 [-0.85, 0.0, 0.968, -0.89],
 [0.775, 0.711, 0.0, 0.742],
 [0.751, 0.0, 0.0, 0.716],
 [0.0, 0.072, 0.0, 0.092],
 [-0.59, 0.0, 0.753, -0.54],
 [0.0, 0.709, 0.0, 0.819],
 [0.763, 0.0, 0.0, 0.719],
 [0.0, 0.0, 0.0, -0.06],
 [0.066, 0.083, -0.06, 0.09],
 [0.0, 0.0, 0.033, -0.04],
 [0.723, 0.851, 0.0, 0.0],
 [0.0, 0.0, 0.0, -0.08],
 [0.633, -0.79, 0.0, 0.935],
 [0.0, 0.149, 0.015, 0.194],
 [-0.75, 0.0, 0.806, 0.0],
 [-0.08, 0.179, -0.06, 0.096],
 [0.0, 0.892, 0.0, 0.0],
 [0.0, 0.977, 0.0, 0.863],
 [-0.98, 0.0, -0.79, 0.0],
 [0.123, 0.072, -0.07, 0.152],
 [0.8, -0.84, 0.0, 0.868],
 [-0.1, -0.03, 0.057, -0.05],
 [0.0, 0.848, 0.0, 0.711],
 [0.501, -0.99, 0.0, 0.68],
 [-0.8, 0.0, 0.945

In [90]:
from collections import defaultdict
ksenticnets = defaultdict(dict)
for key, val, senti, p, seman in zip(keys, 
                                     senticvals, 
                                     sentiments, 
                                     polarity, 
                                     semantics):
    ksenticnets[key]['sentic_value'] = val
    ksenticnets[key]['sentiment'] = senti
    ksenticnets[key]['polarity'] = p
    ksenticnets[key]['semantic'] = seman

In [92]:
ksenticnets['가난뱅이']

{'sentic_value': [-0.05, 0.0, -0.11, -0.04],
 'sentiment': ['#sadness', '#fear'],
 'polarity': ['negative', '-0.06'],
 'semantic': ['빈자']}

In [97]:
sentiwordknu.shape

(428, 6)

In [165]:
'{:7.3f}'.format(-0.7564)

' -0.756'

In [167]:
for i, j, k in zip(keys, senticvals, sentiments):
    print('{:>8s}\t[{:6.3f}, {:6.3f}, {:6.3f}, {:6.3f}]\t[{:11s}, {:11s}]'.format(
        i, *j, *k))

      가게	[ 0.000,  0.124, -0.050,  0.203]	[#interest  , #admiration]
      가격	[-0.003,  0.236,  0.009,  0.147]	[#interest  , #admiration]
      가계	[ 0.000,  0.822,  0.000,  0.712]	[#interest  , #admiration]
      가곡	[ 0.045,  0.000,  0.033,  0.089]	[#joy       , #admiration]
      가공	[ 0.000,  0.921,  0.000,  0.993]	[#interest  , #admiration]
      가교	[ 0.791,  0.857,  0.000,  0.000]	[#joy       , #interest  ]
      가구	[ 0.514, -0.970,  0.000,  0.899]	[#surprise  , #admiration]
      가금	[-0.900,  0.000, -0.874,  0.000]	[#sadness   , #fear      ]
      가난	[-0.373,  0.000,  0.140, -0.911]	[#disgust   , #sadness   ]
    가난뱅이	[-0.050,  0.000, -0.110, -0.040]	[#sadness   , #fear      ]
     가난하	[ 0.000,  0.000, -0.855, -0.755]	[#fear      , #disgust   ]
      가내	[-0.850,  0.000,  0.968, -0.890]	[#anger     , #disgust   ]
      가담	[ 0.775,  0.711,  0.000,  0.742]	[#joy       , #admiration]
     가담하	[ 0.751,  0.000,  0.000,  0.716]	[#joy       , #admiration]
      가도	[ 0.000,  0.072,  0.000, 

      사상	[-0.880,  0.000,  0.983,  0.000]	[#sadness   , #anger     ]
      사색	[ 0.080,  0.153, -0.120,  0.224]	[#interest  , #admiration]
      사선	[ 0.575, -0.620,  0.000,  0.596]	[#surprise  , #admiration]
     사설하	[ 0.617, -0.830,  0.000,  0.828]	[#surprise  , #admiration]
      사수	[ 0.141, -0.120, -0.060, -0.060]	[#joy       , #surprise  ]
      사슴	[ 0.996,  0.000,  0.000,  0.000]	[#joy       , #joy       ]
      사시	[ 0.481,  0.663, -0.250,  0.240]	[#joy       , #interest  ]
      사실	[-0.050, -0.040,  0.017, -0.050]	[#sadness   , #disgust   ]
      사악	[-0.870,  0.000,  0.530, -0.650]	[#sadness   , #disgust   ]
      사양	[-0.770,  0.000,  0.868,  0.000]	[#sadness   , #anger     ]
      사업	[ 0.050,  0.145,  0.000,  0.174]	[#interest  , #admiration]
     사업가	[ 0.832,  0.000,  0.000,  0.000]	[#joy       , #joy       ]
     사역하	[-0.750,  0.000,  0.000, -0.780]	[#sadness   , #disgust   ]
      사연	[ 0.076,  0.079,  0.027,  0.118]	[#interest  , #admiration]
     사열하	[ 0.901,  0.862,  0.000, 

       춤	[ 0.700, -0.700,  0.000,  0.644]	[#joy       , #surprise  ]
       춥	[ 0.173,  0.106, -0.040,  0.067]	[#joy       , #interest  ]
     충격파	[-0.900,  0.000,  0.825, -0.610]	[#sadness   , #anger     ]
      충고	[ 0.575, -0.835,  0.000,  0.746]	[#surprise  , #admiration]
     충돌하	[-0.640,  0.000,  0.575, -0.654]	[#disgust   , #sadness   ]
      충성	[ 0.997,  0.796,  0.000,  0.890]	[#joy       , #admiration]
      충실	[ 0.839,  0.431,  0.000,  0.948]	[#admiration, #joy       ]
      충전	[ 0.000,  0.109, -0.050,  0.108]	[#interest  , #admiration]
      충절	[ 0.997,  0.796,  0.000,  0.890]	[#joy       , #admiration]
      취기	[ 0.000,  0.000,  0.943, -0.800]	[#anger     , #disgust   ]
     취득하	[-0.050,  0.106,  0.000,  0.060]	[#interest  , #admiration]
      취락	[ 0.756, -0.980,  0.000,  0.744]	[#joy       , #surprise  ]
      취미	[ 0.558,  0.661, -0.270,  0.302]	[#interest  , #joy       ]
      취사	[ 0.000,  0.260, -0.130,  0.236]	[#interest  , #admiration]
      취소	[ 0.890, -0.620,  0.000, 

```
+ ;     [joy     , interest  , anger  , admiration]
- ;     [sadness , surprise  , fear   , disgust   ]
```

# Data Preprocessing

In [169]:
sentiwordknu.head()

,일련번호,단어,감정범주,빈도,감정정도M,감정정도STD
0,1,가관이다,혐오,47.5,4.95,2.46
1,2,가련하다,슬픔,88.8,5.62,2.36
2,3,가뿐하다,기쁨,81.3,5.36,2.89
3,4,가소롭다,혐오,46.3,4.82,1.81
4,5,가슴아프다,슬픔,93.8,7.62,2.14


In [171]:
sentiwordknu.T.head()

,0,1,2,3,4,5,6,7,8,9,...,418,419,420,421,422,423,424,425,426,427
일련번호,1,2,3,4,5,6,7,8,9,10,...,419,420,421,422,423,424,425,426,427,428
단어,가관이다,가련하다,가뿐하다,가소롭다,가슴아프다,가슴앓이,가엾다,각박하다,각성되다,간절하다,...,흑흑,흠칫하다,흡족하다,흥겹다,흥미롭다,흥분하다,희희낙락하다,히죽거리다,히죽거리다,히히
감정범주,혐오,슬픔,기쁨,혐오,슬픔,슬픔,슬픔,슬픔,중성,슬픔,...,슬픔,놀람,기쁨,기쁨,흥미,분노,기쁨,기쁨,기쁨,기쁨
빈도,47.5,88.8,81.3,46.3,93.8,90,87.5,42.5,32.5,57.5,...,95,81.3,92.5,93.8,82.5,56.3,95,83.8,83.8,95
감정정도M,4.95,5.62,5.36,4.82,7.62,7.75,6.26,5.87,8.35,4.95,...,6.54,6.79,7.18,7.72,8.99,6.72,7.14,5.61,5.61,5.76
